# We split data into 3 groups for different use cases:
- we will do 20% clustering, 80% train/test/validation

we create 2 csvs
- clustering_data.csv
- train_test_validate_data.csv

Output of this notebook will go into notebook:
- clustering_exploration_v2.ipynb (in the same folder, data_processing)

The groups found from clustering will go into 2 folders: ideal_icu and assigned_icu
- los_prediction_ideal_icu.ipynb, los_prediction_assigned_icu.ipynb
- priors_trueicu_ideal_icu.ipynb, priors_trueicu_assigned_icu.ipynb

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import os

In [3]:
#os.chdir('/Users/chloemaine/Documents/Chloe/BGSE/masters_project/processed_data')

In [3]:
data = pd.read_csv('../output_data/cleaned_data_ideal_and_assigned.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
len(data)

61532

In [64]:
data

,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,ethnicity_grouped,admission_type,...,unclassified,FIRST_CAREUNIT,INTIME,INTIME_COLLAPSED,OUTTIME_COLLAPSED,age_group,dod_exists,sepsis,diagnoses_count,ideal_ICU
0,55973.0,152234.0,200001,F,2182-11-26 00:00:00,2181-11-18 16:04:00,2181-12-04 13:42:00,16.0,asian,EMERGENCY,...,2.0,MICU,2181-11-25 19:06:12,2011-09-04 19:06:12 UTC,2011-09-07 20:59:23 UTC,60.0-79.0,True,False,20.0,MICU
1,27513.0,163557.0,200003,M,2199-10-17 00:00:00,2199-08-02 17:02:00,2199-08-22 19:00:00,20.0,white,EMERGENCY,...,0.0,SICU,2199-08-02 19:50:04,2007-07-13 19:50:04 UTC,2007-07-19 17:09:21 UTC,40.0-59.0,True,True,6.0,SICU
2,10950.0,189514.0,200006,M,2160-05-14 00:00:00,2159-09-03 11:27:00,2159-09-05 18:45:00,2.0,other,EMERGENCY,...,1.0,MICU,2159-09-03 11:28:14,2004-11-22 11:28:14 UTC,2004-11-23 19:08:10 UTC,40.0-59.0,True,False,8.0,MICU
3,20707.0,129310.0,200007,M,NaN,2109-02-17 10:02:00,2109-02-20 15:47:00,3.0,white,EMERGENCY,...,1.0,CCU,2109-02-17 10:03:37,2001-07-22 10:03:37 UTC,2001-07-23 17:03:13 UTC,40.0-59.0,False,False,7.0,CCU
4,29904.0,129607.0,200009,F,NaN,2189-11-30 10:45:00,2189-12-06 15:00:00,6.0,white,ELECTIVE,...,1.0,CSRU,2189-11-30 10:34:32,2006-11-20 10:34:32 UTC,2006-11-22 14:17:35 UTC,40.0-59.0,False,False,6.0,CSRU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,13620.0,169431.0,299993,M,NaN,2149-11-13 21:00:00,2149-11-28 14:30:00,15.0,white,EMERGENCY,...,0.0,SICU,2149-11-13 21:01:05,2004-04-01 21:01:05 UTC,2004-04-04 16:57:26 UTC,20.0-39.0,False,False,2.0,SICU
61528,10718.0,177406.0,299994,F,NaN,2114-12-03 18:07:00,2114-12-26 16:15:00,23.0,white,EMERGENCY,...,0.0,SICU,2114-12-09 11:56:00,2001-12-09 11:56:00 UTC,2001-12-15 20:29:12 UTC,60.0-79.0,False,True,19.0,MICU
61529,28775.0,134959.0,299995,M,NaN,2116-03-04 17:43:00,2116-03-07 16:00:00,3.0,white,EMERGENCY,...,1.0,SICU,2116-03-04 17:44:39,2002-01-16 17:44:39 UTC,2002-01-18 18:03:56 UTC,20.0-39.0,False,False,5.0,MICU
61530,69587.0,158288.0,299998,M,2183-02-25 00:00:00,2181-07-05 07:15:00,2181-08-03 16:03:00,29.0,white,ELECTIVE,...,0.0,MICU,2181-07-05 18:47:40,2011-09-01 18:47:40 UTC,2011-09-03 17:24:34 UTC,80-100,True,True,13.0,MICU


In [24]:
data = data.rename(columns={"INTIME_COLLAPSEDic_": "INTIME_COLLAPSED", 'OUTTIME_COLLAPSEDic_' : 'OUTTIME_COLLAPSED'})


In [56]:
pd.DataFrame(data[data['ideal_ICU']=='NICU']['los_icu'].value_counts()/len(data[data['ideal_ICU']=='NICU'])).sort_index()




,los_icu
0.0,0.421482
1.0,0.140824
2.0,0.044301
3.0,0.022151
4.0,0.018438
...,...
149.0,0.000124
151.0,0.000247
167.0,0.000124
169.0,0.000124


In [25]:
data['age_group'].unique()

array(['60.0-79.0', '40.0-59.0', '20.0-39.0', '80-100', '0.0-19.0',
       'nan-nan'], dtype=object)

In [46]:
len(data[data['age_group'] =='40.0-59.0'])

14684

In [58]:
pd.DataFrame(data[data['age_group'] =='40.0-59.0']['los_icu'].value_counts())



,los_icu
1.0,5200
2.0,3265
3.0,1739
4.0,983
5.0,602
...,...
77.0,1
154.0,1
53.0,1
62.0,1


In [60]:
pd.DataFrame(data[data['age_group'] =='60.0-79.0']['los_icu'].value_counts()/len(data[data['age_group'] =='0.0-19.0'])).sort_index()




,los_icu
0.0,0.047868
1.0,0.855090
2.0,0.558380
3.0,0.336976
4.0,0.192778
...,...
97.0,0.000119
98.0,0.000119
99.0,0.000119
102.0,0.000119


# We choose INTIME_TRANS_COLLAPSED column to make temporal data split 
#### source: Predicting inpatient flow at a major hospital using interpretable analytics
https://web.mit.edu/jpauph/www/pdf/lengthofstay.pdf

They use temporal train/test/validate split. Took 6 months, then 2 months, then 2 months


- We have roughly 12 years of data (total days is datetime.timedelta(days=4226))

In [5]:
# Extracts day from intime col
def extract_day(df, intime_cols): 
    for col in intime_cols:
        col_name = str(col) + "_day"
        df.loc[:, str(col)] = pd.to_datetime(df.loc[:, str(col)])
        df.insert(df.shape[1], str(col_name), df.loc[:,str(col)].dt.date, True) 
    return(df)

In [6]:
def prepare_df(df):
    #turn INTIME_TRANS_COLLAPSED column into type(datetime day)
    df = extract_day(df, ['INTIME_COLLAPSED']).drop(['INTIME_COLLAPSED'], axis=1)
    
    # sort df using INTIME_TRANS_COLLAPSED column, to get indexes for data splits
    df = df.sort_values(by='INTIME_COLLAPSED_day')
    return df


In [7]:
def return_split_data(df):

    bound = int(data.shape[0] * 0.2) #get first 20% of rows
    bound2 = data.shape[0] - bound #rest of 80% for all train/test/validate
    
    df = df.sort_values(by='INTIME_COLLAPSED_day')
    df = df.reset_index()
    clustering_index = [x for x in range(0, bound)]
    train_test_validate_index = [x for x in range(bound, bound+ bound2)]
    print('summed rows', len(clustering_index) + len(train_test_validate_index))
    clustering_data = df.loc[clustering_index] 
    train_test_validate_data = df.loc[train_test_validate_index]
    
    #additional info
    #total number of days we have in data is 4226
    total_days = df['INTIME_COLLAPSED_day'].max() - df['INTIME_COLLAPSED_day'].min()
    print('total days in data', total_days)

    return clustering_data, train_test_validate_data
    
    
    

In [8]:
#clean df and sort
data = prepare_df(data)

In [9]:
data.shape

(61532, 45)

In [10]:
#data.to_csv('practicum_icu_data.csv')

In [11]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'dod', 'admittime',
       'dischtime', 'los_hospital', 'ethnicity_grouped', 'admission_type',
       'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'los_icu',
       'icustay_seq', 'first_icu_stay', 'category', 'first_category', 'blood',
       'circulatory', 'congenital', 'digestive', 'genitourinary',
       'ill-defined', 'infectious|parasitic', 'injury|poisoning', 'mental',
       'metabolic|immunity', 'musculoskeletal', 'neoplasms', 'nervous',
       'perinatal', 'pregnancy complications', 'respiratory', 'skin',
       'unclassified', 'FIRST_CAREUNIT', 'INTIME', 'OUTTIME_COLLAPSED',
       'age_group', 'dod_exists', 'sepsis', 'diagnoses_count', 'ideal_ICU',
       'INTIME_COLLAPSED_day'],
      dtype='object')

In [12]:
data.groupby('INTIME_COLLAPSED_day').size()

INTIME_COLLAPSED_day
2000-12-31     9
2001-01-01    11
2001-01-02    12
2001-01-03     7
2001-01-04     5
              ..
2012-11-23     2
2012-11-29     1
2012-12-03     1
2012-12-07     1
2012-12-29     1
Length: 4264, dtype: int64

In [13]:
#total number of days in our dataset is 4381 days (but we only have 4264 rows because of the days we have zero admissions)

data.groupby('INTIME_COLLAPSED_day').count().index.max() - data.groupby('INTIME_COLLAPSED_day').count().index.min()

datetime.timedelta(days=4381)

In [19]:
## We delete rows where intime date is outside of 1 June 2001 and 10 October 2012.
#from MIMIC-III
#The ADMISSIONS table defines all HADM_ID present in the database, 
#covering an admission period between 1 June 2001 and 10 October 2012.
# from datetime import datetime
# min_date = '2001-06-01'
# min_date = datetime.strptime(min_date, '%Y-%m-%d').date()

# max_date = '2012-10-10'
# max_date = datetime.strptime(max_date, '%Y-%m-%d').date()

In [20]:
# data[data['INTIME_COLLAPSED_day']<min_date] #2202 rows
# data[data['INTIME_COLLAPSED_day']>max_date] #38 rows

# total_rows_deleted = len(data[data['INTIME_COLLAPSED_day']<min_date]) + len(data[data['INTIME_COLLAPSED_day']>max_date])
# total_rows_deleted


In [66]:
# data = data[(data['INTIME_COLLAPSED_day']>min_date) & (data['INTIME_COLLAPSED_day']<max_date)]

In [72]:
data.groupby('INTIME_COLLAPSED_day').count().index.max() - data.groupby('INTIME_COLLAPSED_day').count().index.min()

datetime.timedelta(days=4147)

# We Split Data into 20% clustering, 80% train/test/validate/make priors

In [14]:
#getting split up data
#20% clustering, #80% train, test, validate

clustering_data, train_test_validate_data = return_split_data(data)

summed rows 61532
total days in data 4381 days, 0:00:00


In [15]:
#checks
clustering_data.head()

len(train_test_validate_data)

49226

In [16]:
print(data.INTIME_COLLAPSED_day.max())
print(data.INTIME_COLLAPSED_day.min())
print(data.INTIME_COLLAPSED_day.max() - data.INTIME_COLLAPSED_day.min())

2012-12-29
2000-12-31
4381 days, 0:00:00


In [17]:
print(train_test_validate_data.INTIME_COLLAPSED_day.max())
print(train_test_validate_data.INTIME_COLLAPSED_day.min())
print(train_test_validate_data.INTIME_COLLAPSED_day.max() - train_test_validate_data.INTIME_COLLAPSED_day.min())

2012-12-29
2003-03-01
3591 days, 0:00:00


In [18]:
#write to csv
clustering_data.to_csv('../exploration_data/clustering_data.csv', index=False)
train_test_validate_data.to_csv('../output_data/train_test_validate_data.csv', index=False)
